In [ ]:
#-----------------------------------------------------------------------
# context를 가지고, 바드 혹은 GPT를 이용하여, sLLM 파인튜닝할 질의응답 말뭉치를 만드는 예시임.
# -> context 문단들을 수동으로 만들고, 이를 이용하여 바드 혹은 gpt에게, 아래처럼 프롬프트를 만들어서 쿼리하여, 응답을 가지고 말뭉치로 만듬.
# 아래 ##문단:을 가지고 10개의 질문과 답변을 ##질문:, ##답변: 쌍 형식으로 만들어주세요. 단 ##질문: 뒤에 숫자는 붙이면 안됩니다.\n\n##문단:{context}"
# -> 
#
#-----------------------------------------------------------------------
import os
import random
import numpy as np
import pandas as pd
import time
import random
from tqdm.notebook import tqdm
import sys
from myutils import seed_everything, GPU_info, mlogging

SEED = 111
seed_everything(SEED)
DEVICE = GPU_info() # GPU 혹은 CPU
LOGGER = mlogging(loggername="kopora-test", logfilename='../log/kopora-test.txt') # 로그

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

files = ["../data11/mpower_doc/moco_context.txt",] # 컨텍스트 문단들 파일
count = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.

tcontexts = []
tcontextids = []

for idx, file_path in enumerate(files):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        data_list = data.split('\n\n')  # '\n\n' 구분으로 다락 구분
        count += 1
        
        # titles, contextids 임의로 구해서 데이터 프레임 만듬.
        for i in range(len(data_list)):
            tcontexts.append(data_list[i])
            tcontextids.append(count)

# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(tcontexts, tcontextids)), columns = ['context','contextid'])     

print(f'len:{len(df_contexts)}\n')

In [ ]:
df_contexts['context'][0]

In [ ]:
def make_prompt(context):
    #prompt = f'아래 ##문단:을 가지고 10개의 질문과 답변을 ##질문:, ##답변: 쌍 형식으로 만들어주세요. 단 ##질문: 뒤에 숫자는 붙이면 안됩니다.\n\n##문단:{context}'
    prompt = f'아래 ##문단 내용이 질문에 대한 답변일대, ##질문 문장을 만들고, 아래 문단 내용을 요약해서 ##답변 문장을 만들어 주세요.\n\n{context}'
    return prompt

In [ ]:
#------------------------------------------------------------------
# 구글 bard를 이용한 text 생성
#
# 세션키를 이용하여 구글 bard 테스트 예제
# 출처 : https://github.com/dsdanielpark/Bard-API
#
# token 값얻기
# https://bard.google.com/ 방문
# 콘솔용 F12
# 세션: 애플리케이션 → 쿠키 → 쿠키 값을 복사합니다 __Secure-1PSID.
# -> 참고로 반드시 뒤에 .으로 끝나고 .포함해서 길이가 72자임.
#------------------------------------------------------------------
import json
from bardapi import Bard
token = 'XXX' # bard 토큰 입력

def generate_text_BARD(prompt):
    bard = Bard(token=token,timeout=30) # Set timeout in seconds
    answer = bard.get_answer(prompt)['content']
    return answer
    

In [ ]:
#-----------------------------------------
# GPT를 이용한 text 생성
#-----------------------------------------
import openai

# key 지정
openai.api_key = "sk-xxx"

# 모델 - GPT 3.5 Turbo 지정
# => 모델 목록은 : https://platform.openai.com/docs/models/gpt-4 참조
gpt_model = "gpt-3.5-turbo"#"gpt-4"#"gpt-3.5-turbo" #gpt-4-0314

# 메시지 설정
MESSAGES:list = []

#-----------------------------------------
# GPT를 이용한 text 생성
#-----------------------------------------
def generate_text_GPT(prompt, messages):
    
    #print(f'len(messages):{len(messages)}') 
    #print()
    
    #-------------------------------------------------------
    # *** gpt에 메시지는 계속 대화 내용이 유지가 되므로, 비용이 발생함.
    # 따라서 최근 2개 대화만 유지함.
    #if len(messages) >= 2:
    #    messages = messages[len(messages)-2:]  # 최근 2개의 대화만 가져오기
    messages = []  # 무조건 최근대화 초기화
     #-------------------------------------------------------
        
    # 사용자 메시지 추가
    messages.append( {"role": "user", "content": prompt})
    #print(messages)

    # ChatGPT-API 호출하기
    response = openai.ChatCompletion.create(
        model=gpt_model,
        messages=messages,
        max_tokens=1024, # 토큰 수 
        temperature=1,  # temperature 0~2 범위 : 작을수록 정형화된 답변, 클수록 유연한 답변(2는 엉뚱한 답변을 하므로, 1.5정도가 좋은것 같음=기본값은=1)
        top_p=0.1 # 기본값은 1 (0.1이라고 하면 10% 토큰들에서 출력 토큰들을 선택한다는 의미)
    )

    #print(response)
    #print()
    answer = response['choices'][0]['message']['content']
    return answer


In [ ]:
contextids = df_contexts['contextid'].values.tolist()
contexts = df_contexts['context'].values.tolist()

examples = []
for context in tqdm(contexts[50:]):
    #time.sleep(1)
    #print(context)
    
    # prompt 생성
    prompt = make_prompt(context)
    #print(prompt)
    #print()
    
    answer = generate_text_BARD(prompt) # bard 이용할때
    #answer = generate_text_GPT(prompt=prompt, messages=MESSAGES) #chat gpt이용할때
    #print(answer)
    answer_list = answer.split('\n\n')  # '\n\n' 구분으로 다락 구분
    
    doc = {}
    doc['context'] = context
    doc['qa'] = answer_list
                     
    examples.append(doc)

In [ ]:
len(examples)

In [ ]:
examples[0]

In [ ]:
# 텍스트로 파일에 저장
with open('./data/moco_qa_60.txt', 'w') as f:

    # 리스트를 파일에 씁니다.
    for item in examples:
        #print(item['context'])
        #print(item['qa'])
        f.write(item['context'] + '\n\n')
        for qa in item['qa']:
            f.write(qa + '\n')
            
        f.write('\n\n')

In [ ]:
# *이후 파일을 열어서, 수동으로 아래처럼 수정함.
# -> contexts 들은 \n\n\n 으로 구분함.
# -> ##질문: {} \n ##답변:{} 은 쌍으로 하고, 다른 질문/답변과는 \n\n 구분함.

In [ ]:
# 조작할 파일들을 불러와서 contexts별로 분할
count = 0
file_path = './data/moco_qa_gpt.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    data = f.read()
    docs = data.split('\n\n\n')  # '\n\n\n' 구분으로 contexts 단락 구분
    count += 1

In [ ]:
from tqdm.notebook import tqdm
contexts = []
questions = []
answers = []

for doc in tqdm(docs):
    data1 = doc.split('\n\n')
    qa_list = data1[1:]

    for qa in qa_list:
        data2 = qa.split('##답변:')
        if len(data2) == 2:
            questions.append(data2[0].replace("##질문:", ""))
            answers.append(data2[1].replace("##답변:", ""))
            contexts.append(data1[0])
        

In [ ]:
print(contexts[1000])
print()
print(answers[1000])
print()
print(questions[1000])
print()

print(f'{len(contexts)} == {len(questions)} == {len(answers)}')


In [ ]:
import json
examples = []

for context, question, answer in zip(contexts, questions, answers):
    doc = {}
    doc['question'] = question
    doc['context'] = context
    doc['answer'] = answer
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("./data/moco_qa_summerize_gpt.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)
    